<a href="https://colab.research.google.com/github/pastrop/kaggle/blob/master/data_prep_sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import numpy as np
import os

In [7]:
# file upload while using Google Colab
from google.colab import files
uploaded = files.upload()

Saving train_data.dat to train_data.dat


In [ ]:
!unzip raw_data.zip

In [5]:
# Load Dataframe Test Set: assume nomad.csv as input file name
df = pd.read_csv('raw-data.csv',  encoding="latin_1")
df.head(3)

,doc.id,title,citeulike.id,raw.title,raw.abstract
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,..."


In [39]:
df_final=df.drop(columns=['title', 'citeulike.id'])
df_final['combined_text'] = df_final['raw.title'].str.cat(df_final['raw.abstract'],sep=" ")

In [44]:
len(df_final['combined_text'][50])

1883

In [13]:
def load_user_article_likes(
    data_folder: str, num_users: int = 5551, num_articles: int = 13584
) -> np.ndarray:
    """
    Load the article likes for each user
    Keyword arguments:
        data_folder -- the path to the folder that contains the data
        num_users -- the number of users in the dataset - default is 5551
        num_articles -- the number of articles in the dataset - default is 13584
    Returns:
        2d numpy matrix with describing user like information.
        This matrix is num_users X num_articles.
        There is a 1 in locations in which a user likes an article u -> a
        citations[u][a] and 0 otherwise.
    """
    user_items = np.zeros([num_users, num_articles])
    with open(os.path.join(data_folder, "train_data.dat")) as users_file:
        for i, line in enumerate(users_file.readlines()):
            for article in line.strip().split()[1:]:
                user_items[i][int(article)] = 1
    return user_items

In [16]:
res = load_user_article_likes('')

In [21]:
res.shape

(5551, 13584)

In [35]:
result_len = []
for i in range(res.shape[0]):
  result_len.append(len(np.where(res[i] == 1)[0]))
max(result_len)

346

In [45]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])

print(embeddings)

tf.Tensor(
[[-0.03133018 -0.06338634 -0.01607502 ... -0.03242778 -0.04575741
   0.05370456]
 [ 0.05080861 -0.01652428  0.01573781 ...  0.00976659  0.03170123
   0.01788119]], shape=(2, 512), dtype=float32)


In [46]:
embeddings.numpy()

array([[-0.03133018, -0.06338634, -0.01607502, ..., -0.03242778,
        -0.04575741,  0.05370456],
       [ 0.05080861, -0.01652428,  0.01573781, ...,  0.00976659,
         0.03170123,  0.01788119]], dtype=float32)